In [ ]:
###############################################################################
import aliases

In [ ]:
from everest.ptolemaic.intt import Intt

In [ ]:
Intt[10:20][-5:][-2]

In [ ]:
from everest.ptolemaic.ptolemaic import Ptolemaic as _Ptolemaic
from everest.ptolemaic.chora import (
    Sliceable as _Sliceable, Incisable as _Incisable
    )
from everest.ptolemaic.chora import NotNone as _NotNone

class MyChora(_Sliceable):

    def _retrieve_contains_(self, incisor: int, /) -> int:
        return incisor**2

    def _incise_slice_(self,
            start: _NotNone, stop: _NotNone, step: _NotNone, /
            ):
        return (start, stop, step)


class MyPtol(_Ptolemaic, _Incisable):

    Chora = MyChora

    def incise(self, incisor, /):
        return f"Incised {self} by {incisor}"

#     def retrieve(self, incisor, /):
#         return f"Retrieved {incisor} from {self}"

In [ ]:
myobj = MyPtol()

In [ ]:
myobj[2:4:6]

In [ ]:
myobj[2]

In [ ]:
myobj = MyChora()

In [ ]:
myobj.chora.__getitem__

In [ ]:
issubclass(int, NotNone)

In [ ]:
issubclass(NotNone, int)

In [ ]:
class MyClass:

    @classmethod
    def __subclasscheck__(cls, other, /):
        print('hello')

In [ ]:
issubclass(int, MyClass)

In [ ]:
issubclass(int, NotNone)

In [ ]:
import abc as _abc


class Null(_abc.ABC):

    @classmethod
    def __subclasscheck__(cls, arg, /):
        return True

In [ ]:
isinstance(1, Null)

In [ ]:
issubclass(int, NotNone)

In [ ]:
myobj[2:3:4]

In [ ]:
class MyClass:

    def __getitem__(self, arg, /):
        '''Placeholder.'''
        print('Foo!')


def __getitem__(self, arg, /):
    return arg**2


myobj = MyClass()
myobj.__getitem__ = __getitem__
myobj[2]